In [ ]:
! pip install numpy pandas matplotlib

In [2]:
"""
1.
The Tabu search search space is assumed to be discrete, even finite. Consider a continuous optimization setting, for example the minimization of a function [0,1] x [0,1] → R. Here the concept of a neighbor of a point is ambiguous as each point has infinitely many neighbors. How would you define the neighbors of a solution so that tabu search could be used?
"""
print()

In [1]:
"""
Answer:
Firstly, I would bin the search space into discrete, evenly spaced search space.
For example in [0,1] x [0,1]  search space. Essentially, I would basically bin it into 11 x 11 search space like binning histogram. This way the search space is discrete and even finite. For more granularity, one could split the search space even further at the price of bigger search space.

Secondly, for each dimension of the search space, the neighbor is basically the increase or decrease of each dimension at one step. For example in [0,1] x [0,1] search space, we turn it into 11 x 11 search space of 0 to 1 with spacing of 0.1 each. If we initialize at [0.1, 0.5], our neighbors would be a list of [0.0,0.5], [0.2,0.5], [0.1,0.6], [0.1,0.4]. This would allow us to respect the new search space we have created with tabu list assumption while keeping the neighbors size limited.

Therefore, we would have a search space of [0, 0.1, 0.2,...., 0.9, 1.0] for each dimension according to example and we can define our neighbors as increase or decrease in each dimension.

However, this method suffers from inaccuracy due to the fact that we are not looking for the actual optima point but its approximation.
"""
print()

In [3]:
"""

2.

Download the code in tabu.py. The aim is to fill in the missing parts of the code in order to solve QAP with tabu search as described in the book chapter. You can implement everything from scratch if you like.

By default, the code solves the QAP instance given on the lecture slides. Once you have working code, solve the given NUG5 instance (the global optimum is 50).

(3 points)

Then uncomment the NUG20 instance and report the best objective value you could obtain.

Try different iteration numbers and tenures. More QAP instances are found in http://mistic.heig-vd.ch/taillard/problemes.dir/qap.dir/qap.html if you are interested.

The file format is as follows:

Format of quadratic assignment problem instances :

Size of the problem, best solution value (if known)
flow matrix
distance matrix
(1 point)
"""
print()

In [ ]:
"""

2.

Download the code in tabu.py. The aim is to fill in the missing parts of the code in order to solve QAP with tabu search as described in the book chapter. You can implement everything from scratch if you like.

By default, the code solves the QAP instance given on the lecture slides. Once you have working code, solve the given NUG5 instance (the global optimum is 50).

(3 points)

Then uncomment the NUG20 instance and report the best objective value you could obtain.

Try different iteration numbers and tenures. More QAP instances are found in http://mistic.heig-vd.ch/taillard/problemes.dir/qap.dir/qap.html if you are interested.

The file format is as follows:

Format of quadratic assignment problem instances :

Size of the problem, best solution value (if known)
flow matrix
distance matrix
(1 point)
"""
print()

# Q2 Implementation

In [2]:
# imports
import numpy as np
import random
from ast import literal_eval as make_tuple


In [3]:
# test make_tuple for long term memory implementation
assert (0,1) == make_tuple(str((0, 1)))

In [4]:
def generate_moves(indices):
    """
    generate all possible index swap
    :param indices:
    :return:
    """
    moves = []
    for i in range(len(indices) - 1):
        for j in range(i + 1, len(indices)):
            if i == j:
                continue
            moves.append((i, j))
    return moves


class QAPTabuLongTermMemory:
    def __init__(self, moves):
        self.moves_frequency_map = {str(move): 0 for move in moves}

    def update_stat(self, move):
        self.moves_frequency_map[str(move)] += 1

    def get_least_frequent_move(self):
        move = min(self.moves_frequency_map, key=self.moves_frequency_map.get)
        return make_tuple(move)


class TabuSearchQAP:
    def __init__(self,
                 kmax: int,
                 kmin: int,
                 flows_matrix,
                 distance_matrix,
                 max_iterations: int):
        assert 0 < kmin < kmax, "kmin must be more than 0 and less than kmax"
        self.distance_matrix = distance_matrix
        self.flows_matrix = flows_matrix
        self.problem_size = len(self.flows_matrix)
        self.tabu_matrix = np.zeros((self.problem_size, self.problem_size))
        # index --> location
        # value --> item
        self.solution = np.arange(self.problem_size)

        np.random.shuffle(self.solution)
        self.moves = generate_moves(self.solution)  # move based on indices
        self.kmax = kmax
        self.kmin = kmin
        # maybe long term memory of moves
        self.long_term_memory = QAPTabuLongTermMemory(self.moves)
        self.fitness = None
        self.max_iterations = max_iterations
        self.best_solution = self.solution
        self.best_fitness = self.get_current_fitness(self.solution)
        self.current_iteration = 0

    def solve(self):
        fitness = self.best_fitness
        for i in range(self.current_iteration, self.max_iterations):
            result = self.run_one_iteration(self.solution, fitness, i)
            if result is None:
                move_chosen = self.long_term_memory.get_least_frequent_move()
                result = self.force_generate_neighbor_and_fitness_with_move(
                    self.solution, fitness, move_chosen)
            move_chosen, new_solution, new_fitness = result
            self.update_tabu_list(move_chosen, i)
            self.long_term_memory.update_stat(move_chosen)
            self.solution = new_solution
            fitness = new_fitness
            if new_fitness < self.best_fitness:
                self.best_solution = new_solution
                self.best_fitness = new_fitness
            self.current_iteration = i

    def force_generate_neighbor_and_fitness_with_move(self, current_solution, current_fitness, move):
        neighbor = np.copy(current_solution)
        neighbor[move[0]], neighbor[move[1]] = neighbor[move[1]], neighbor[move[0]]
        new_fitness = self.get_neighbor_fitness(current_solution,
                                                neighbor,
                                                current_fitness,
                                                move)
        return move, neighbor, new_fitness

    def update_tabu_list(self, move, iteration):
        for m in move:
            item = self.solution[m]
            self.tabu_matrix[item][m] = iteration + random.randint(self.kmin,
                                                                   self.kmax)

    def is_index_less_than_iteration(self, index_from, index_to, iteration) -> bool:
        item = self.solution[index_from]
        # while update_tabu_list add value to move_from of item m
        # tabu checks if the location move_to of the item in move_from
        # is correct
        if self.tabu_matrix[item][index_to] <= iteration:
            return True
        return False

    def is_move_tabu(self, move, iteration) -> bool:
        return not self.is_index_less_than_iteration(move[0], move[1],
                                                     iteration) and not self.is_index_less_than_iteration(
            move[1], move[0], iteration)

    def run_one_iteration(self, current_solution, current_fitness, iteration):
        """
        :param iteration:
        :param current_solution:
        :param current_fitness:
        :return: (move_chosen, new_solution, new_fitness)
        """
        neighbors = []
        neighbors_fitness = []
        moves_chosen = []
        for move in self.moves:
            if self.is_move_tabu(move, iteration):
                continue
            neighbor = np.copy(current_solution)
            neighbor[move[0]], neighbor[move[1]] = neighbor[move[1]], neighbor[move[0]]
            neighbors.append(neighbor)
            neighbors_fitness.append(self.get_neighbor_fitness(current_solution, neighbor, current_fitness, move))
            moves_chosen.append(move)
        moves_and_neighbors_and_fitness = list(zip(moves_chosen, neighbors, neighbors_fitness))
        if len(moves_and_neighbors_and_fitness) == 0:
            return None
        moves_and_neighbors_and_fitness = sorted(moves_and_neighbors_and_fitness, key=lambda x: x[2], reverse=False)
        return moves_and_neighbors_and_fitness[0]

    def get_neighbor_fitness(self, current_solution, neighbor, fitness, move):
        """
        O(n) update algorithm
        You can see that we for each loop we have 2 subtraction/addition instances, this is because in our get_current_fitness, we add both side of the connections. Technically, we can halve our fitnesses function calculation by using only right/left half of the metric to perform their calculation.
        :param current_solution:
        :param neighbor:
        :param fitness:
        :param move:
        :return:
        """
        nfitness = fitness
        # remove old fitness via new solution
        for m in move:
            for i in range(len(current_solution)):
                nfitness -= self.flows_matrix[current_solution[m]][current_solution[i]] * self.distance_matrix[m][i]
                nfitness -= self.flows_matrix[current_solution[i]][current_solution[m]] * self.distance_matrix[i][m]

        # add new fitness via neighbor
        for m in move:
            for i in range(len(neighbor)):
                nfitness += self.flows_matrix[neighbor[m]][neighbor[i]] * self.distance_matrix[m][i]
                nfitness += self.flows_matrix[neighbor[i]][neighbor[m]] * self.distance_matrix[i][m]

        return nfitness

    def get_current_fitness(self, solution) -> int:
        fitness = 0
        # re-calculate entire fitness size
        # basically all combinations of current setup
        for i in range(len(solution)):
            for j in range(len(solution)):
                fitness += self.flows_matrix[solution[i]][solution[j]] * self.distance_matrix[i][j]
        return fitness

In [5]:
# the flow and distances for qap4 from tabu.py line 60-61
# example is from book
F = np.array([0, 1, 0, 1,
              1, 0, 0, 2,
              0, 0, 0, 3,
              1, 2, 3, 0]).reshape(4, 4)
D = np.array([0, 4, 3, 5,
              4, 0, 5, 3,
              3, 5, 0, 4,
              5, 3, 4, 0]).reshape(4, 4)

In [6]:
print(F)
print(D)

[[0 1 0 1]
 [1 0 0 2]
 [0 0 0 3]
 [1 2 3 0]]
[[0 4 3 5]
 [4 0 5 3]
 [3 5 0 4]
 [5 3 4 0]]


In [7]:
"""
kmax: int,
kmin: int,
flows_matrix,
distance_matrix,
max_iterations: int
"""
tsq = TabuSearchQAP(10,
                    3,
                    F,
                    D,
                    24)

In [8]:
tsq.solve()
# A B C D -> locations
# 3 1 2 0 -> items
# [3, 1, 2, 0] -> solution -> fitness 25*2 = 50
# result is multiplied by 2 because we perform the fitness addition on both side of the metric
# since theyre symmetric, we technically only need to do 1 side but the implementation is a little bit cumbersome.
# my update fitness is currently wrong i think
# yup it is --> if put in current fitness i get 50, 58 for the examples
print(tsq.best_solution)
print(tsq.best_fitness)

[0 2 1 3]
50


In [13]:
F = np.array([0,5,2,4,1,
              5,0,3,0,2,
              2,3,0,0,0,
              4,0,0,0,5,
              1,2,0,5,0]).reshape(5, 5)
D = np.array([0,1,1,2,3,
              1,0,2,1,2,
              1,2,0,1,2,
              2,1,1,0,1,
              3,2,2,1,0]).reshape(5, 5)
tsq = TabuSearchQAP(20,
                    10,
                    F,
                    D,
                    1000)
tsq.solve()
# this is correct at 50 since global minima mentioned in the book is 50
print(tsq.solution)
print(tsq.best_solution)
print(tsq.best_fitness)

[1 0 3 2 4]
[3 4 0 1 2]
50


In [22]:
# Here the fitness is 70 not 72
# might need to ask ivan what he means cus i get 72
# sanity checking to follow fitness according to the book
f1 = tsq.get_current_fitness([1,3,0,4,2])
f2 = tsq.get_neighbor_fitness([1,3,0,4,2],
                             [0,3,1,4,2],
                             f1,
                             (0,2))
f3 = tsq.get_neighbor_fitness([0,3,1,4,2],
                             [4,3,1,0,2],
                             f2,
                             (0,3))
f4 = tsq.get_neighbor_fitness([4,3,1,0,2],
                             [4,1,3,0,2],
                             52,
                             (1,2))
print(f1,f2,f3,f4)

72 60 52 52


In [24]:
F = np.array([
    [0, 1, 2, 3, 4, 1, 2, 3, 4, 5, 2, 3, 4, 5, 6, 3, 4, 5, 6, 7],
    [1, 0, 1, 2, 3, 2, 1, 2, 3, 4, 3, 2, 3, 4, 5, 4, 3, 4, 5, 6],
    [2, 1, 0, 1, 2, 3, 2, 1, 2, 3, 4, 3, 2, 3, 4, 5, 4, 3, 4, 5],
    [3, 2, 1, 0, 1, 4, 3, 2, 1, 2, 5, 4, 3, 2, 3, 6, 5, 4, 3, 4],
    [4, 3, 2, 1, 0, 5, 4, 3, 2, 1, 6, 5, 4, 3, 2, 7, 6, 5, 4, 3],
    [1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 1, 2, 3, 4, 5, 2, 3, 4, 5, 6],
    [2, 1, 2, 3, 4, 1, 0, 1, 2, 3, 2, 1, 2, 3, 4, 3, 2, 3, 4, 5],
    [3, 2, 1, 2, 3, 2, 1, 0, 1, 2, 3, 2, 1, 2, 3, 4, 3, 2, 3, 4],
    [4, 3, 2, 1, 2, 3, 2, 1, 0, 1, 4, 3, 2, 1, 2, 5, 4, 3, 2, 3],
    [5, 4, 3, 2, 1, 4, 3, 2, 1, 0, 5, 4, 3, 2, 1, 6, 5, 4, 3, 2],
    [2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 1, 2, 3, 4, 5],
    [3, 2, 3, 4, 5, 2, 1, 2, 3, 4, 1, 0, 1, 2, 3, 2, 1, 2, 3, 4],
    [4, 3, 2, 3, 4, 3, 2, 1, 2, 3, 2, 1, 0, 1, 2, 3, 2, 1, 2, 3],
    [5, 4, 3, 2, 3, 4, 3, 2, 1, 2, 3, 2, 1, 0, 1, 4, 3, 2, 1, 2],
    [6, 5, 4, 3, 2, 5, 4, 3, 2, 1, 4, 3, 2, 1, 0, 5, 4, 3, 2, 1],
    [3, 4, 5, 6, 7, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4],
    [4, 3, 4, 5, 6, 3, 2, 3, 4, 5, 2, 1, 2, 3, 4, 1, 0, 1, 2, 3],
    [5, 4, 3, 4, 5, 4, 3, 2, 3, 4, 3, 2, 1, 2, 3, 2, 1, 0, 1, 2],
    [6, 5, 4, 3, 4, 5, 4, 3, 2, 3, 4, 3, 2, 1, 2, 3, 2, 1, 0, 1],
    [7, 6, 5, 4, 3, 6, 5, 4, 3, 2, 5, 4, 3, 2, 1, 4, 3, 2, 1, 0]
])
D = np.array([
    [0,  0,  5,  0,  5,  2, 10,  3,  1,  5,  5,  5,  0,  0,  5,  4,  4,  0,  0,  1],
    [0,  0,  3, 10,  5,  1,  5,  1,  2,  4,  2,  5,  0, 10, 10,  3,  0,  5, 10,  5],
    [5,  3,  0,  2,  0,  5,  2,  4,  4,  5,  0,  0,  0,  5,  1,  0,  0,  5,  0,  0],
    [0, 10,  2,  0,  1,  0,  5,  2,  1,  0, 10,  2,  2,  0,  2,  1,  5,  2,  5,  5],
    [5,  5,  0,  1,  0,  5,  6,  5,  2,  5,  2,  0,  5,  1,  1,  1,  5,  2,  5,  1],
    [2,  1,  5,  0,  5,  0,  5,  2,  1,  6,  0,  0, 10,  0,  2,  0,  1,  0,  1,  5],
    [10,  5,  2,  5,  6,  5,  0,  0,  0,  0,  5, 10,  2,  2,  5,  1,  2,  1,  0, 10],
    [3,  1,  4,  2,  5,  2,  0,  0,  1,  1, 10, 10,  2,  0, 10,  2,  5,  2,  2, 10],
    [1,  2,  4,  1,  2,  1,  0,  1,  0,  2,  0,  3,  5,  5,  0,  5,  0,  0,  0,  2],
    [5,  4,  5,  0,  5,  6,  0,  1,  2,  0,  5,  5,  0,  5,  1,  0,  0,  5,  5,  2],
    [5,  2,  0, 10,  2,  0,  5, 10,  0,  5,  0,  5,  2,  5,  1, 10,  0,  2,  2,  5],
    [5,  5,  0,  2,  0,  0, 10, 10,  3,  5,  5,  0,  2, 10,  5,  0,  1,  1,  2,  5],
    [0,  0,  0,  2,  5, 10,  2,  2,  5,  0,  2,  2,  0,  2,  2,  1,  0,  0,  0,  5],
    [0, 10,  5,  0,  1,  0,  2,  0,  5,  5,  5, 10,  2,  0,  5,  5,  1,  5,  5,  0],
    [5, 10,  1,  2,  1,  2,  5, 10,  0,  1,  1,  5,  2,  5,  0,  3,  0,  5, 10, 10],
    [4,  3,  0,  1,  1,  0,  1,  2,  5,  0, 10,  0,  1,  5,  3,  0,  0,  0,  2,  0],
    [4,  0,  0,  5,  5,  1,  2,  5,  0,  0,  0,  1,  0,  1,  0,  0,  0,  5,  2,  0],
    [0,  5,  5,  2,  2,  0,  1,  2,  0,  5,  2,  1,  0,  5,  5,  0,  5,  0,  1,  1],
    [0, 10,  0,  5,  5,  1,  0,  2,  0,  5,  2,  2,  0,  5, 10,  2,  2,  1,  0,  6],
    [1,  5,  0,  5,  1,  5, 10, 10,  2,  2,  5,  5,  5,  0, 10,  0,  0,  1,  6,  0]
])

In [26]:
tsq = TabuSearchQAP(50,
                    10,
                    F,
                    D,
                    10000)
tsq.solve()
print(tsq.best_solution)
print(tsq.best_fitness)

[ 3 11 18 10  1  4  2  8 19 17 13 12  9 16  6 14  0 15  5  7]
2570


In [30]:
# QUestion 3 - read .dat file
def read_qap_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Read the size of the problem instance
    l1 = lines[0].strip().split()
    problem_size, best_solution_value = int(l1[0]), None if len(l1) == 1 else tuple(map(int, l1))

    # Read the distance matrix
    distance_matrix = np.zeros((problem_size, problem_size))
    for i in range(2, problem_size + 2):
        row = list(map(int, lines[i].strip().split()))
        distance_matrix[i - 2] = row

    # Read the flow matrix
    flow_matrix = np.zeros((problem_size, problem_size))
    for i in range(problem_size + 3, 2 * problem_size + 3):
        row = list(map(int, lines[i].strip().split()))
        flow_matrix[i - problem_size - 3] = row

    return problem_size, distance_matrix, flow_matrix


In [31]:
problem_size, F, D = read_qap_instance('nug24.dat')
tsq = TabuSearchQAP(100,
                    10,
                    F,
                    D,
                    10000)
tsq.solve()
print(tsq.best_solution)
print(tsq.best_fitness)

[ 3 23 19 18 17 16 14  1 20 15 13 10 22  6 11 12  5  8  2  0 21  9  7  4]
3490.0
